In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')
import time, os
from pandas.io.html import read_html
from pandas import DataFrame

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [2]:
# List of team stats url

epl_stats_url = [
["2009", "https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/1849/England-Premier-League", ' "tournament-tables-3115" '],
["2010", "https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/2458/England-Premier-League", ' "tournament-tables-4345" '],
["2011", "https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/2935/England-Premier-League", ' "tournament-tables-5476" '],
["2012", "https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/3389/England-Premier-League", ' "tournament-tables-6531" '],
["2013", "https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/3853/England-Premier-League", ' "tournament-tables-7794" '],
["2014", "https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/4311/England-Premier-League", ' "tournament-tables-9155" '],
["2015", "https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/5826/England-Premier-League", ' "tournament-tables-12496" '],
["2016", "https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/6335/England-Premier-League", ' "tournament-tables-13796" '],
["2017", "https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/6829/England-Premier-League", ' "tournament-tables-15151" '],
["2018", "https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/7361/England-Premier-League", ' "tournament-tables-16368" '],                               
["2019", "https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/7811/England-Premier-League", ' "tournament-tables-17590" '],
["2020", "https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/8228/England-Premier-League", ' "tournament-tables-18685" ']
]

laliga_stats_url = [
["2009", "https://www.whoscored.com/Regions/206/Tournaments/4/Seasons/1929/Spain-LaLiga", ' "tournament-tables-3218" '],
["2010", "https://www.whoscored.com/Regions/206/Tournaments/4/Seasons/2596/Spain-LaLiga", ' "tournament-tables-4624" '],
["2011", "https://www.whoscored.com/Regions/206/Tournaments/4/Seasons/3004/Spain-LaLiga", ' "tournament-tables-5577" '],
["2012", "https://www.whoscored.com/Regions/206/Tournaments/4/Seasons/3470/Spain-LaLiga", ' "tournament-tables-6652" '],
["2013", "https://www.whoscored.com/Regions/206/Tournaments/4/Seasons/3922/Spain-LaLiga", ' "tournament-tables-7920" '],
["2014", "https://www.whoscored.com/Regions/206/Tournaments/4/Seasons/5435/Spain-LaLiga", ' "tournament-tables-11363" '],
["2015", "https://www.whoscored.com/Regions/206/Tournaments/4/Seasons/5933/Spain-LaLiga", ' "tournament-tables-12647" '],
["2016", "https://www.whoscored.com/Regions/206/Tournaments/4/Seasons/6436/Spain-LaLiga", ' "tournament-tables-13955" '],
["2017", "https://www.whoscored.com/Regions/206/Tournaments/4/Seasons/6960/Spain-LaLiga", ' "tournament-tables-15375" '],
["2018", "https://www.whoscored.com/Regions/206/Tournaments/4/Seasons/7466/Spain-LaLiga", ' "tournament-tables-16546" '],
["2019", "https://www.whoscored.com/Regions/206/Tournaments/4/Seasons/7889/Spain-LaLiga", ' "tournament-tables-17702" '],
["2020", "https://www.whoscored.com/Regions/206/Tournaments/4/Seasons/8321/Spain-LaLiga", ' "tournament-tables-18851" ']
]

bundesliga_stats_url = [
["2009", "https://www.whoscored.com/Regions/81/Tournaments/3/Seasons/1903/Germany-Bundesliga", ' "tournament-tables-3181" '],
["2010", "https://www.whoscored.com/Regions/81/Tournaments/3/Seasons/2520/Germany-Bundesliga", ' "tournament-tables-4448" '],
["2011", "https://www.whoscored.com/Regions/81/Tournaments/3/Seasons/2949/Germany-Bundesliga", ' "tournament-tables-5492" '],
["2012", "https://www.whoscored.com/Regions/81/Tournaments/3/Seasons/3424/Germany-Bundesliga", ' "tournament-tables-6576" '],
["2013", "https://www.whoscored.com/Regions/81/Tournaments/3/Seasons/3863/Germany-Bundesliga", ' "tournament-tables-7806" '],
["2014", "https://www.whoscored.com/Regions/81/Tournaments/3/Seasons/4336/Germany-Bundesliga", ' "tournament-tables-9192" '],
["2015", "https://www.whoscored.com/Regions/81/Tournaments/3/Seasons/5870/Germany-Bundesliga", ' "tournament-tables-12559" '],
["2016", "https://www.whoscored.com/Regions/81/Tournaments/3/Seasons/6392/Germany-Bundesliga", ' "tournament-tables-13872" '],
["2017", "https://www.whoscored.com/Regions/81/Tournaments/3/Seasons/6902/Germany-Bundesliga", ' "tournament-tables-15243" '],
["2018", "https://www.whoscored.com/Regions/81/Tournaments/3/Seasons/7405/Germany-Bundesliga", ' "tournament-tables-16427" '],
["2019", "https://www.whoscored.com/Regions/81/Tournaments/3/Seasons/7872/Germany-Bundesliga", ' "tournament-tables-17682" '],
["2020", "https://www.whoscored.com/Regions/81/Tournaments/3/Seasons/8279/Germany-Bundesliga", ' "tournament-tables-18762" ']
]

seriea_stats_url = [
["2009", "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/1957/Italy-Serie-A", ' "tournament-tables-3277" '],
["2010", "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/2626/Italy-Serie-A", ' "tournament-tables-4659" '],
["2011", "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/3054/Italy-Serie-A", ' "tournament-tables-5667" '],
["2012", "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/3512/Italy-Serie-A", ' "tournament-tables-6739" '],
["2013", "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/3978/Italy-Serie-A", ' "tournament-tables-8019" '],
["2014", "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/5441/Italy-Serie-A", ' "tournament-tables-11369" '],
["2015", "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/5970/Italy-Serie-A", ' "tournament-tables-12770" '],
["2016", "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/6461/Italy-Serie-A", ' "tournament-tables-14014" '],
["2017", "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/6974/Italy-Serie-A", ' "tournament-tables-15404" '],
["2018", "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/7468/Italy-Serie-A", ' "tournament-tables-16548" '],
["2019", "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/7928/Italy-Serie-A", ' "tournament-tables-17835" '],
["2020", "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/8330/Italy-Serie-A", ' "tournament-tables-18873" ']
]

ligue1_stats_url = [
["2009", "https://www.whoscored.com/Regions/74/Tournaments/22/Seasons/1839/France-Ligue-1", ' "tournament-tables-3099" '],
["2010", "https://www.whoscored.com/Regions/74/Tournaments/22/Seasons/2417/France-Ligue-1", ' "tournament-tables-4273" '],
["2011", "https://www.whoscored.com/Regions/74/Tournaments/22/Seasons/2920/France-Ligue-1", ' "tournament-tables-5451" '],
["2012", "https://www.whoscored.com/Regions/74/Tournaments/22/Seasons/3356/France-Ligue-1", ' "tournament-tables-6476" '],
["2013", "https://www.whoscored.com/Regions/74/Tournaments/22/Seasons/3836/France-Ligue-1", ' "tournament-tables-7771" '],
["2014", "https://www.whoscored.com/Regions/74/Tournaments/22/Seasons/4279/France-Ligue-1", ' "tournament-tables-9105" '],
["2015", "https://www.whoscored.com/Regions/74/Tournaments/22/Seasons/5830/France-Ligue-1", ' "tournament-tables-12501" '],
["2016", "https://www.whoscored.com/Regions/74/Tournaments/22/Seasons/6318/France-Ligue-1", ' "tournament-tables-13768" '],
["2017", "https://www.whoscored.com/Regions/74/Tournaments/22/Seasons/6833/France-Ligue-1", ' "tournament-tables-15155" '],
["2018", "https://www.whoscored.com/Regions/74/Tournaments/22/Seasons/7344/France-Ligue-1", ' "tournament-tables-16348" '],
["2019", "https://www.whoscored.com/Regions/74/Tournaments/22/Seasons/7814/France-Ligue-1", ' "tournament-tables-17593" '],
["2020", "https://www.whoscored.com/Regions/74/Tournaments/22/Seasons/8185/France-Ligue-1", ' "tournament-tables-18594" ']
]

# links to club values
value_url = ['https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/saison_id/',
              'https://www.transfermarkt.com/primera-division/startseite/wettbewerb/ES1/saison_id/',
              'https://www.transfermarkt.com/1-bundesliga/startseite/wettbewerb/L1/saison_id/',
              'https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1/saison_id/',
              'https://www.transfermarkt.com/ligue-1/startseite/wettbewerb/FR1/saison_id/']

In [3]:
# Create empty dictionary for leagues

epl_dict = {}        # English Premier League
laliga_dict = {}     # Spanish La Liga
bundesliga_dict = {} # German Bundes Liga
seriea_dict = {}     # Italian Serie A
ligue1_dict = {}     # French Ligue 1

# Create empty DataFrame for years 1999 to 2020 in each league

for yr in range(2009,2021):     # 15 years
    epl_dict['epl_df_%s' % yr] = pd.DataFrame
    laliga_dict['laliga_df_%s' % yr] = pd.DataFrame 
    bundesliga_dict['bundesliga_df_%s' % yr] = pd.DataFrame 
    seriea_dict['seriea_df_%s' % yr] = pd.DataFrame 
    ligue1_dict['ligue1_df_%s' % yr] = pd.DataFrame
    
# List of league dictionaries

leagues_list = [epl_dict,laliga_dict,bundesliga_dict,seriea_dict,ligue1_dict] 

In [4]:
# Scrape transfermarkt.com and whoscored.com to gather soccer data for analysis

for league in range(0,len(value_url)):
    time.sleep(3)
    for yr in range(2009,2021):
        time.sleep(3)
        #--------------------------
        # Scrape transfermakt.com
        #--------------------------        
        driver = webdriver.Chrome(chromedriver)
        url = value_url[league] + str(yr)
        #url = 'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/saison_id/2020'
        driver.get(url)

        table_df = pd.DataFrame()
        team_value_df = pd.DataFrame()

        time.sleep(3)

        table = driver.find_element_by_xpath('//*[@class="responsive-table"]')
        table_html = table.get_attribute('innerHTML')
        df1 = read_html(table_html)[0]
        team_value_df = pd.concat([team_value_df, df1])

        time.sleep(3)

        table = driver.find_element_by_xpath('//*[@class="box tab-print"]')
        table_html = table.get_attribute('innerHTML')
        df2 = read_html(table_html)[0]
        table_df = pd.concat([table_df, df2])

        driver.close()
        
        team_value_df.dropna(subset=["club.1"], axis=0, inplace=True)
        team_value_df = team_value_df.drop(["club","ø market value","Total MV","ø MV"], axis=1, errors="ignore")
        team_value_df.rename(columns={'club.1':'CLUB', 'name':'TEAM', 'name.1':'SQUAD', 'Squad': 'AVG_AGE',\
                'ø age':'FOREIGNERS', 'Foreigners':'TOTAL_MARKET_VALUE (€)','Total market value':'AVG_MARKET_VALUE (€)'},inplace=True)

        table_df = table_df.drop(["club","Matches","+/-","Pts"], axis=1, errors="ignore")
        table_df.rename(columns={'#':'POSITION', 'club.1':'TEAM'},inplace=True)

        table_value_df = table_df.merge(team_value_df, on='TEAM')
        table_value_df = table_value_df.drop(["TEAM"], axis=1, errors="ignore")
        table_value_df['YEAR'] = yr # League season

        #--------------------------
        # Scrape whoscored.com
        #--------------------------

        if league == 0:
            url_data = epl_stats_url
        elif league == 1:
            url_data = laliga_stats_url
        elif league == 2:
            url_data = bundesliga_stats_url
        elif league == 3:
            url_data = seriea_stats_url
        else:
            url_data = ligue1_stats_url
            
        time.sleep(3)

        tmp = yr - 2009 # Set index to access rows of league stats url
        driver = webdriver.Chrome(chromedriver)
        url = url_data[tmp][1]
        #url = 'https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/8228/Stages/18685/Show/England-Premier-League-2020-2021'
        driver.get(url)

        standings_df = DataFrame()
        summary_df = DataFrame()
        defensive_df = DataFrame()
        offensive_df = DataFrame()
        #detailed_df = DataFrame() # Returns empty document when scraped

        tmp1 = '//*[@id='+ url_data[tmp][2] + ']'
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH,tmp1)))
        time.sleep(20)
        #table = driver.find_element_by_xpath('//*[@id="tournament-tables-18685"]')
        table = driver.find_element_by_xpath(tmp1)
        table_html = table.get_attribute('innerHTML')
        time.sleep(10)
        df1 = read_html(table_html)[0]
        standings_df = pd.concat([standings_df, df1])

        time.sleep(10)
        team_statistics_button = driver.find_element_by_xpath("//*[contains(text(), 'Team Statistics')]")
        team_statistics_button.click()
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH,'//*[@id="statistics-team-table-summary"]')))
        time.sleep(20)
        table = driver.find_element_by_xpath('//*[@id="statistics-team-table-summary"]')
        table_html = table.get_attribute('innerHTML')
        time.sleep(10)
        df2 = read_html(table_html)[0]
        summary_df = pd.concat([summary_df, df2])

        defensive_button = driver.find_element_by_xpath("//*[contains(text(), 'Defensive')]")
        defensive_button.click()
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH,'//*[@id="statistics-team-table-defensive"]')))
        time.sleep(20)
        table = driver.find_element_by_xpath('//*[@id="statistics-team-table-defensive"]')
        table_html = table.get_attribute('innerHTML')
        time.sleep(10)
        df3 = read_html(table_html)[0]
        defensive_df = pd.concat([defensive_df, df3])

        offensive_button = driver.find_element_by_xpath("//*[contains(text(), 'Offensive')]")
        offensive_button.click()
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH,'//*[@id="statistics-team-table-offensive"]')))
        time.sleep(20)
        table = driver.find_element_by_xpath('//*[@id="statistics-team-table-offensive"]')
        table_html = table.get_attribute('innerHTML')
        time.sleep(10)
        df4 = read_html(table_html)[0]
        offensive_df = pd.concat([offensive_df, df4])

        '''
        detailed_button = driver.find_element_by_xpath("//*[contains(text(), 'Detailed')]")
        detailed_button.click()
        time.sleep(20)
        table = driver.find_element_by_xpath('//*[@id="statistics-team-table-detailed"]')
        table_html = table.get_attribute('innerHTML')
        df4 = read_html(table_html)[0]
        defensive_df = pd.concat([detailed_df, df4])
        '''

        driver.close()

        standings_df = standings_df.drop(["Form"], axis=1, errors="ignore")
        standings_df['Team'] = standings_df.Team.str.strip('1234567890. ')

        summary_df['Discipline'] = summary_df.Discipline.astype('str')
        summary_df['YC'] = [x[0:2] for x in summary_df['Discipline']] 
        summary_df['RC'] = [x[-1] for x in summary_df['Discipline']]
        summary_df = summary_df.drop(["Discipline","Rating","Goals"], axis=1, errors="ignore")

        offensive_df = offensive_df.drop(["Shots pg","Rating"], axis=1, errors="ignore")

        defensive_df = defensive_df.drop(["Rating"], axis=1, errors="ignore")
        defensive_df.rename(columns={'Shots pg':'SHOTS_PG_DEF'},inplace=True)

        stats_df = summary_df.merge(offensive_df, on='Team').merge(defensive_df,on='Team')
        stats_df['Team'] = stats_df.Team.str.strip('1234567890. ')

        standings_stats_df = standings_df.merge(stats_df, on='Team')

        standings_stats_df['Team'] = table_value_df['CLUB']
        standings_stats_df.rename(columns={'Team':'CLUB'},inplace=True)
        #table_value_stats_df = table_value_df.merge(standings_stats_df, on='CLUB')

        if league == 0:
            leagues_list[league]['epl_df_%s' % yr] = table_value_df.merge(standings_stats_df, on='CLUB')
        elif league == 1:
            leagues_list[league]['laliga_df_%s' % yr] = table_value_df.merge(standings_stats_df, on='CLUB')    
        elif league == 2:
            leagues_list[league]['bundesliga_df_%s' % yr] = table_value_df.merge(standings_stats_df, on='CLUB') 
        elif league == 3:
            leagues_list[league]['seriea_df_%s' % yr] = table_value_df.merge(standings_stats_df, on='CLUB') 
        else:
            leagues_list[league]['ligue1_df_%s' % yr] = table_value_df.merge(standings_stats_df, on='CLUB')
            
        print(f'Completed: league ---> {league} ::::: year ---> {yr}')


Completed: league ---> 0 ::::: year ---> 2009
Completed: league ---> 0 ::::: year ---> 2010
Completed: league ---> 0 ::::: year ---> 2011
Completed: league ---> 0 ::::: year ---> 2012
Completed: league ---> 0 ::::: year ---> 2013
Completed: league ---> 0 ::::: year ---> 2014
Completed: league ---> 0 ::::: year ---> 2015
Completed: league ---> 0 ::::: year ---> 2016
Completed: league ---> 0 ::::: year ---> 2017
Completed: league ---> 0 ::::: year ---> 2018
Completed: league ---> 0 ::::: year ---> 2019
Completed: league ---> 0 ::::: year ---> 2020
Completed: league ---> 1 ::::: year ---> 2009
Completed: league ---> 1 ::::: year ---> 2010
Completed: league ---> 1 ::::: year ---> 2011
Completed: league ---> 1 ::::: year ---> 2012
Completed: league ---> 1 ::::: year ---> 2013
Completed: league ---> 1 ::::: year ---> 2014
Completed: league ---> 1 ::::: year ---> 2015
Completed: league ---> 1 ::::: year ---> 2016
Completed: league ---> 1 ::::: year ---> 2017
Completed: league ---> 1 ::::: yea

In [14]:
# Combine dataframes for each year.

epl_df_concat = pd.concat(epl_dict.values(),ignore_index=True)
laliga_df_concat = pd.concat(laliga_dict.values(),ignore_index=True)
bundesliga_df_concat = pd.concat(bundesliga_dict.values(),ignore_index=True)
seriea_df_concat = pd.concat(seriea_dict.values(),ignore_index=True)
ligue1_df_concat = pd.concat(ligue1_dict.values(),ignore_index=True)

In [15]:
# Add a country column

epl_df_concat['COUNTRY'] = "England"
laliga_df_concat['COUNTRY'] = "Spain"
bundesliga_df_concat['COUNTRY'] = "Germany"
seriea_df_concat['COUNTRY'] = "Italy"
ligue1_df_concat['COUNTRY'] = "France"

# Combined leagues dataframe

leagues_df = pd.concat([epl_df_concat,laliga_df_concat,bundesliga_df_concat,seriea_df_concat,ligue1_df_concat], \
                       ignore_index=True)

In [19]:
leagues_df.columns

Index(['POSITION', 'CLUB', 'SQUAD', 'AVG_AGE', 'FOREIGNERS',
       'TOTAL_MARKET_VALUE (€)', 'AVG_MARKET_VALUE (€)', 'YEAR', 'P', 'W', 'D',
       'L', 'GF', 'GA', 'GD', 'Pts', 'Shots pg', 'Possession%', 'Pass%',
       'AerialsWon', 'YC', 'RC', 'Shots OT pg', 'Dribbles pg', 'Fouled pg',
       'SHOTS_PG_DEF', 'Tackles pg', 'Interceptions pg', 'Fouls pg',
       'Offsides pg', 'COUNTRY'],
      dtype='object')

In [20]:
# Rename Columns

leagues_df.rename(columns={'TOTAL_MARKET_VALUE (€)':'TOTAL_MARKET_VALUE (m€)',\
                           'AVG_MARKET_VALUE (€)':'AVG_MARKET_VALUE (m€)','Pts':'POINTS',\
                           'Shots pg':'SHOTS_PG', 'Possession%':'POSSESSION_PERCENT',\
                           'Pass%':'PASS_PERCENT', 'AerialsWon':'AERIALS_WON',\
                           'Shots OT pg':'SHOTS_OT_PG', 'Dribbles pg':'DRIBBLES_PG',\
                           'Fouled pg':'FOULED_PG', 'Tackles pg':'TACKLES_PG',\
                           'Interceptions pg':'INTERCEPTIONS_PG','Fouls pg':'FOULS_PG',\
                           'Offsides pg':'OFFSIDES_PG'},inplace=True)

In [23]:
leagues_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1176 entries, 0 to 1175
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   POSITION                 1176 non-null   int64  
 1   CLUB                     1176 non-null   object 
 2   SQUAD                    1176 non-null   float64
 3   AVG_AGE                  1176 non-null   float64
 4   FOREIGNERS               1176 non-null   object 
 5   TOTAL_MARKET_VALUE (m€)  1176 non-null   object 
 6   AVG_MARKET_VALUE (m€)    1176 non-null   object 
 7   YEAR                     1176 non-null   int64  
 8   P                        1176 non-null   int64  
 9   W                        1176 non-null   int64  
 10  D                        1176 non-null   int64  
 11  L                        1176 non-null   int64  
 12  GF                       1176 non-null   int64  
 13  GA                       1176 non-null   int64  
 14  GD                      

In [24]:
leagues_df.head()

,POSITION,CLUB,SQUAD,AVG_AGE,FOREIGNERS,TOTAL_MARKET_VALUE (m€),AVG_MARKET_VALUE (m€),YEAR,P,W,...,RC,SHOTS_OT_PG,DRIBBLES_PG,FOULED_PG,SHOTS_PG_DEF,TACKLES_PG,INTERCEPTIONS_PG,FOULS_PG,OFFSIDES_PG,COUNTRY
0,1,Chelsea FC,33.0,25.5,24,€443.90m,€13.45m,2009,38,27,...,5,7.6,8.6,12.3,9.8,20.3,18.6,12.4,2.5,England
1,2,Manchester United,33.0,25.0,22,€362.95m,€11.00m,2009,38,27,...,4,5.8,8.7,11.7,10.9,21.8,15.8,11.7,2.2,England
2,3,Arsenal FC,45.0,21.7,31,€281.00m,€6.24m,2009,38,23,...,1,6.1,11.9,14.0,10.1,21.5,22.1,12.4,2.4,England
3,4,Tottenham Hotspur,41.0,23.3,20,€249.20m,€6.08m,2009,38,21,...,1,5.9,9.0,13.2,13.7,22.2,21.2,12.2,3.1,England
4,5,Manchester City,40.0,24.1,24,€314.90m,€7.87m,2009,38,18,...,2,5.1,7.7,12.8,13.5,21.1,21.2,11.9,3.2,England


In [25]:
# Edit total market value and market value columns

leagues_df['TOTAL_MARKET_VALUE (m€)'] = leagues_df['TOTAL_MARKET_VALUE (m€)']. \
    apply(lambda x: (x+'0').replace('.','') if 'bn' in x else x).str.replace('bn','').str.replace('€','')    
leagues_df['TOTAL_MARKET_VALUE (m€)'] = leagues_df['TOTAL_MARKET_VALUE (m€)']. \
    apply(lambda x: '0.' + x if 'Th.' in x else x).str.replace('Th.','').str.replace('m','').str.replace('€','')

leagues_df['AVG_MARKET_VALUE (m€)'] = leagues_df['AVG_MARKET_VALUE (m€)']. \
    apply(lambda x: (x+'0').replace('.','') if 'bn' in x else x).str.replace('bn','').str.replace('€','')    
leagues_df['AVG_MARKET_VALUE (m€)'] = leagues_df['AVG_MARKET_VALUE (m€)']. \
    apply(lambda x: '0.' + x if 'Th.' in x else x).str.replace('Th.','').str.replace('m','').str.replace('€','')

In [26]:
leagues_df.head()

,POSITION,CLUB,SQUAD,AVG_AGE,FOREIGNERS,TOTAL_MARKET_VALUE (m€),AVG_MARKET_VALUE (m€),YEAR,P,W,...,RC,SHOTS_OT_PG,DRIBBLES_PG,FOULED_PG,SHOTS_PG_DEF,TACKLES_PG,INTERCEPTIONS_PG,FOULS_PG,OFFSIDES_PG,COUNTRY
0,1,Chelsea FC,33.0,25.5,24,443.90,13.45,2009,38,27,...,5,7.6,8.6,12.3,9.8,20.3,18.6,12.4,2.5,England
1,2,Manchester United,33.0,25.0,22,362.95,11.00,2009,38,27,...,4,5.8,8.7,11.7,10.9,21.8,15.8,11.7,2.2,England
2,3,Arsenal FC,45.0,21.7,31,281.00,6.24,2009,38,23,...,1,6.1,11.9,14.0,10.1,21.5,22.1,12.4,2.4,England
3,4,Tottenham Hotspur,41.0,23.3,20,249.20,6.08,2009,38,21,...,1,5.9,9.0,13.2,13.7,22.2,21.2,12.2,3.1,England
4,5,Manchester City,40.0,24.1,24,314.90,7.87,2009,38,18,...,2,5.1,7.7,12.8,13.5,21.1,21.2,11.9,3.2,England


In [28]:
# Champions in 2020-2021 season

leagues_df.loc[(leagues_df['POSITION']==1) & (leagues_df['YEAR']==2020)]

,POSITION,CLUB,SQUAD,AVG_AGE,FOREIGNERS,TOTAL_MARKET_VALUE (m€),AVG_MARKET_VALUE (m€),YEAR,P,W,...,RC,SHOTS_OT_PG,DRIBBLES_PG,FOULED_PG,SHOTS_PG_DEF,TACKLES_PG,INTERCEPTIONS_PG,FOULS_PG,OFFSIDES_PG,COUNTRY
220,1,Manchester City,25.0,26.7,19,1040,41.74,2020,38,27,...,2,5.7,12.8,9.8,7.4,13.1,8.4,9.5,1.5,England
460,1,Atlético Madrid,22.0,27.8,17,760.00,34.55,2020,38,26,...,0,4.9,9.8,11.7,9.5,16.4,10.2,12.9,2.5,Spain
678,1,Bayern Munich,26.0,26.1,13,826.60,31.79,2020,34,24,...,3,6.9,12.9,11.0,10.1,15.0,11.7,9.3,2.2,Germany
916,1,Inter Milan,25.0,28.9,16,648.90,25.96,2020,38,28,...,2,5.4,6.8,11.2,10.2,13.3,9.8,12.4,1.7,Italy
1156,1,LOSC Lille,23.0,25.4,15,316.80,13.77,2020,38,24,...,2,4.8,10.7,11.8,8.9,17.2,11.2,12.8,1.7,France


In [31]:
# Convert column datatypes for computation

leagues_df['FOREIGNERS'] = leagues_df['FOREIGNERS'].astype('int')
leagues_df['TOTAL_MARKET_VALUE (m€)'] = leagues_df['TOTAL_MARKET_VALUE (m€)'].astype('float')
leagues_df['AVG_MARKET_VALUE (m€)'] = leagues_df['AVG_MARKET_VALUE (m€)'].astype('float')
leagues_df['YC'] = leagues_df['YC'].astype('int')
leagues_df['RC'] = leagues_df['RC'].astype('int')

In [32]:
leagues_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1176 entries, 0 to 1175
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   POSITION                 1176 non-null   int64  
 1   CLUB                     1176 non-null   object 
 2   SQUAD                    1176 non-null   float64
 3   AVG_AGE                  1176 non-null   float64
 4   FOREIGNERS               1176 non-null   int64  
 5   TOTAL_MARKET_VALUE (m€)  1176 non-null   float64
 6   AVG_MARKET_VALUE (m€)    1176 non-null   float64
 7   YEAR                     1176 non-null   int64  
 8   P                        1176 non-null   int64  
 9   W                        1176 non-null   int64  
 10  D                        1176 non-null   int64  
 11  L                        1176 non-null   int64  
 12  GF                       1176 non-null   int64  
 13  GA                       1176 non-null   int64  
 14  GD                      

In [33]:
# Pickle current dataframe for further analysis.

import pickle

In [34]:
# Save leagues_df as pickle in pwd
# Go to project_2_regression_new.ipynb for analysis

with open('leagues_df.pickle', 'wb') as to_write:
    pickle.dump(leagues_df, to_write)